# STT -> LLM -> TTS Test Space

pipeline and stack:

* STT: coqui tts, vosk (which one?)
* LLM: ollama, langchain
* TTS: coqui tts
* AUDIO I/O: pyaudio, sounddevice

### Audio I/O Testing

In [2]:
# audio IO - pyaudio test (playback - sample)
import wave
import sys
import pyaudio

chunksize = 1024
f = 'output.wav'

with wave.open(f, 'rb') as wf:
    # Instantiate PyAudio and initialize PortAudio system resources (1)
    p = pyaudio.PyAudio()

    # Open steam (2)
    stream = p.open(format=p.get_format_from_width(wf.getsampwidth()),
                    channels=wf.getnchannels(),
                    rate=wf.getframerate(),
                    output=True)

    # Play samples from the wave file (3)
    while len(data := wf.readframes(chunksize)):
        stream.write(data)

    # Close stream (4)
    stream.close()

    # Release PortAudio system resources (5)
    p.terminate()

In [3]:
# audio IO - pyaudio test (record)
import wave
import sys
import pyaudio
import math

chunksize = 1024
f = 'record.wav'
seconds = 5
rate = 44100
channels = 1
form = pyaudio.paInt16

# Instantiate PyAudio and initialize PortAudio system resources (1)
p = pyaudio.PyAudio()

# Open steam (2)
stream = p.open(format=form,
                channels=channels,
                rate=rate,
                input=True,
                frames_per_buffer=chunksize)

# instantiate frames container
print ("recording started")
recordframes = []

# record w/ logic for seconds
for i in range(0, math.ceil(rate / chunksize * seconds)):
    data = stream.read(chunksize)
    recordframes.append(data)
print ("recording stopped")
stream.stop_stream()

# Close stream (4)
stream.close()

# Release PortAudio system resources (5)
p.terminate()

# wav file
wf = wave.open(f, 'wb')
wf.setnchannels(channels)
wf.setsampwidth(p.get_sample_size(form))
wf.setframerate(rate)
wf.writeframes(b''.join(recordframes))
wf.close()

recording started
recording stopped


In [4]:
# audio IO - pyaudio test (playback - sample)
import wave
import sys
import pyaudio

chunksize = 1024
f = 'record.wav'

with wave.open(f, 'rb') as wf:
    # Instantiate PyAudio and initialize PortAudio system resources (1)
    p = pyaudio.PyAudio()

    # Open steam (2)
    stream = p.open(format=p.get_format_from_width(wf.getsampwidth()),
                    channels=wf.getnchannels(),
                    rate=wf.getframerate(),
                    output=True)

    # Play samples from the wave file (3)
    while len(data := wf.readframes(chunksize)):
        stream.write(data)

    # Close stream (4)
    stream.close()

    # Release PortAudio system resources (5)
    p.terminate()

### Voice Synthesis Testing

In [5]:
import torch
from TTS.api import TTS
from datetime import date 

script = 'Hey fryman, pass me the peanut butter'

# Get device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Init TTS
tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2").to(device)

# Text to speech to a file
tts.tts_to_file(text=script, speaker_wav="wav_training/p1.wav", language="en", file_path=f"wav_sample/test_p1_{date.today().strftime('%Y%m%d%H%M%S')}.wav")

 > tts_models/multilingual/multi-dataset/xtts_v2 is already downloaded.
 > Using model: xtts
 > Text splitted to sentences.
['Hey fryman, pass me the peanut butter']
 > Processing time: 2.366482734680176
 > Real-time factor: 0.6703615660290067


'wav_sample/test_p1_20250319000000.wav'

### LLM Instantiation Testing

In [1]:
# instantiate ollama - is this necessary when running win app?
import os
os.system('ollama run llama3.2')

0

In [1]:
from ollama import chat
from ollama import ChatResponse

In [3]:
# demo example - https://github.com/ollama/ollama-python

response: ChatResponse = chat(model='llama3.2', messages=[
  {
    'role': 'user',
    'content': 'Why is the sky blue?',
  },
])
print(response['message']['content'])
# or access fields directly from the response object
print(response.message.content)

The sky appears blue because of a phenomenon called Rayleigh scattering, named after the British physicist Lord Rayleigh. This process occurs when sunlight enters Earth's atmosphere and interacts with tiny molecules of gases such as nitrogen (N2) and oxygen (O2).

Here's what happens:

1. Sunlight is made up of different colors, each with its own unique wavelength.
2. When sunlight passes through the atmosphere, it encounters tiny molecules of gases like N2 and O2.
3. The smaller molecules scatter the shorter wavelengths (like blue and violet) more than the longer wavelengths (like red and orange). This scattering effect is known as Rayleigh scattering.
4. As a result, the scattered blue light is dispersed in all directions, reaching our eyes from every part of the sky.
5. Our eyes perceive this dispersed blue light as the color of the sky.

The reason why the sky appears blue during the daytime and not at sunrise or sunset is because:

* During sunrise and sunset, the sun's rays have 

In [4]:
type(response)

ollama._types.ChatResponse

In [6]:
response

ChatResponse(model='llama3.2', created_at='2025-05-07T01:40:34.096695Z', done=True, done_reason='stop', total_duration=2903853900, load_duration=24000700, prompt_eval_count=31, prompt_eval_duration=2999400, eval_count=334, eval_duration=2876853800, message=Message(role='assistant', content="The sky appears blue because of a phenomenon called Rayleigh scattering, named after the British physicist Lord Rayleigh. This process occurs when sunlight enters Earth's atmosphere and interacts with tiny molecules of gases such as nitrogen (N2) and oxygen (O2).\n\nHere's what happens:\n\n1. Sunlight is made up of different colors, each with its own unique wavelength.\n2. When sunlight passes through the atmosphere, it encounters tiny molecules of gases like N2 and O2.\n3. The smaller molecules scatter the shorter wavelengths (like blue and violet) more than the longer wavelengths (like red and orange). This scattering effect is known as Rayleigh scattering.\n4. As a result, the scattered blue ligh

In [8]:
response['message']

Message(role='assistant', content="The sky appears blue because of a phenomenon called Rayleigh scattering, named after the British physicist Lord Rayleigh. This process occurs when sunlight enters Earth's atmosphere and interacts with tiny molecules of gases such as nitrogen (N2) and oxygen (O2).\n\nHere's what happens:\n\n1. Sunlight is made up of different colors, each with its own unique wavelength.\n2. When sunlight passes through the atmosphere, it encounters tiny molecules of gases like N2 and O2.\n3. The smaller molecules scatter the shorter wavelengths (like blue and violet) more than the longer wavelengths (like red and orange). This scattering effect is known as Rayleigh scattering.\n4. As a result, the scattered blue light is dispersed in all directions, reaching our eyes from every part of the sky.\n5. Our eyes perceive this dispersed blue light as the color of the sky.\n\nThe reason why the sky appears blue during the daytime and not at sunrise or sunset is because:\n\n* 